# 0. DataSetting

In [1]:
#import packages & libraries
import pandas as pd
import os
import glob
from sklearn.model_selection import StratifiedKFold
from scipy.ndimage import rotate
import numpy as np
import pydicom
from skimage import measure
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import h5py

In [2]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU 가속 사용 가능")
else:
    device = torch.device("cpu")
    print("GPU 가속 사용 불가능")

# 간단한 연산을 GPU에서 실행
a = torch.tensor([1.0, 2.0, 3.0], device=device)
b = torch.tensor([4.0, 5.0, 6.0], device=device)
c = a + b

print("연산 결과:", c)

GPU 가속 사용 가능
연산 결과: tensor([5., 7., 9.], device='cuda:0')


In [2]:
num_cores = os.cpu_count()
print(f"Number of available cores: {num_cores}")

Number of available cores: 48


In [3]:
num_gpus = torch.cuda.device_count()
print(f"Number of available GPUs: {num_gpus}")

NameError: name 'torch' is not defined

## 0-1 Ground Truh Label Setting

In [2]:
# 주 데이터 폴더 경로
main_data_folder = "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast"
image_data_folder = "/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/201~300/201~300_screen"

# 세부 폴더의 이름
sub_folder_name = '201~300'

# 개별 환자 폴더 설정
patient_folder_names = [f for f in os.listdir(image_data_folder) if os.path.isdir(os.path.join(image_data_folder, f))]

# Find the first Excel file in the directory
excel_file_path = glob.glob(f"{main_data_folder}/{sub_folder_name}/*.xlsx")[0]
excel_data = pd.read_excel(excel_file_path)

labels = excel_data['Molecular subtype(1 : luminal A, 2 : luminal B, 3: Her2 positive, 4: Triple negative)'].values

## 0-2 Roi Dataset Setting

In [4]:
# 병변을 나타내는 색상들의 RGB 값
lesion_colors = np.array([
    [255, 0, 0],      # Red
    [255, 255, 0],    # Yellow1
    [255, 102, 0],    # Yellow2
    [0, 102, 255],   # Blue1
    [0, 0, 255],     # Blue2
    [0, 255, 0],    # Green
    [128, 0, 255]   # Purple
])

# DICOM 파일들을 읽고 이미지 데이터를 가져오는 함수
def load_patient_images(patient_folder):
    
    # patient_folder 내의 DICOM 파일들을 가져오기
    dicom_files = [os.path.join(patient_folder, f) for f in os.listdir(patient_folder) if f.endswith('.dcm')]

    # DICOM 파일들을 읽기
    dicoms = [pydicom.dcmread(d) for d in dicom_files]

    # 각 DICOM 파일의 이미지 위치 정보를 기준으로 정렬
    dicoms.sort(key=lambda d: float(d.ImagePositionPatient[2]))

    # 각 DICOM 파일의 pixel_array를 가져오기
    images = np.stack([d.pixel_array for d in dicoms])

    return images

# 병변 영역을 찾는 함수
def find_lesions(images):
    lesion_mask = np.any([np.all(images == color, axis=-1) for color in lesion_colors], axis=0)
    return lesion_mask

# 가장 큰 병변의 Bounding Volume을 찾는 함수
def find_largest_lesion_and_bounding_volume(lesion_mask):
    
    # 3D Connected Component Labeling 수행
    labels = measure.label(lesion_mask)

    # 각 병변의 크기를 계산하고 가장 큰 병변을 찾기
    sizes = np.bincount(labels.ravel())
    max_label = sizes[1:].argmax() + 1  # 0 레이블은 배경이므로 제외

    # 가장 큰 병변에 해당하는 픽셀들의 좌표를 찾기
    lesion_pixels = np.where(labels == max_label)

    # bounding volume의 모서리를 찾기
    min_corner = np.array([np.min(i) for i in lesion_pixels])
    max_corner = np.array([np.max(i) for i in lesion_pixels])

    return min_corner, max_corner

# DICOM 파일들을 읽고 이미지 데이터를 가져오는 함수
def load_dicoms(patient_folder):
    
    # patient_folder 내의 DICOM 파일들을 가져오기
    dicom_files = [os.path.join(patient_folder, f) for f in os.listdir(patient_folder) if f.endswith('.dcm')]

    # DICOM 파일들을 읽기
    dicoms = [pydicom.dcmread(d) for d in dicom_files]

    # 각 DICOM 파일의 이미지 위치 정보를 기준으로 정렬
    dicoms.sort(key=lambda d: float(d.ImagePositionPatient[2]))

    # 각 DICOM 파일의 pixel_array를 가져오기
    images = np.stack([d.pixel_array for d in dicoms])

    return images, dicoms

# images를 3D 복셀로 변환합니다.
def create_voxel(images, dicoms):
    
    # 모든 2D DICOM 이미지를 하나의 3D numpy 배열로 결합
    voxel = images.astype(float)

    # DICOM 파일의 RescaleIntercept와 RescaleSlope를 적용하여 픽셀 값 조정
    voxel *= np.float64(dicoms[0].RescaleSlope)
    voxel += np.float64(dicoms[0].RescaleIntercept)

    return voxel

In [5]:
def process_patient_folder(patient_folder_name):
    try:
        print(f"Processing folder: {patient_folder_name}")
        
        patient_folder_path = os.path.join(image_data_folder, patient_folder_name) # 환자 폴더 경로
        folder_list = os.listdir(patient_folder_path)

        # 환자의 일련번호를 patient_num 변수에 저장
        patient_num = int(patient_folder_name)

        # 'cad'와 'precontrast' 폴더 찾기
        cad_folder_name = next(f for f in folder_list if 'cad' in f.lower())
        precontrast_folder_name = next(f for f in folder_list if 'precontrast' in f.lower())

        # 경로 생성
        cad_folder = os.path.join(patient_folder_path, cad_folder_name)
        precontrast_folder = os.path.join(patient_folder_path, precontrast_folder_name)

        # 각 폴더의 하위 폴더 찾기
        cad_subfolder_name = next(f for f in os.listdir(cad_folder) if os.path.isdir(os.path.join(cad_folder, f)))
        precontrast_subfolder_name = next(f for f in os.listdir(precontrast_folder) if os.path.isdir(os.path.join(precontrast_folder, f)))

        # DICOM 폴더 경로 생성
        dicom_folder = os.path.join(cad_folder, cad_subfolder_name)
        dicom_folder_pre = os.path.join(precontrast_folder, precontrast_subfolder_name)

        # DICOM 이미지들을 로드하기
        images = load_patient_images(dicom_folder)

        # 병변 마스크를 생성하기
        lesion_mask = find_lesions(images)

        # 가장 큰 병변의 Bounding Volume 찾기
        min_corner, max_corner = find_largest_lesion_and_bounding_volume(lesion_mask)

        # DICOM 이미지를 로드하기
        images_pre, dicoms_pre = load_dicoms(dicom_folder_pre)

        # images_pre를 3D 복셀로 변환합니다.
        voxels_pre = create_voxel(images_pre, dicoms_pre)

        # precontrast MRI에서 해당 ROI 잘라내기
        roi_precontrast = voxels_pre[min_corner[0]:max_corner[0], min_corner[1]:max_corner[1], min_corner[2]:max_corner[2]]

        return patient_num, roi_precontrast
    
    except Exception as e:
        print(f"Failed processing folder: {patient_folder_name}")
        raise e
# HDF5 파일로 저장하기 위한 h5py.File 인스턴스 생성
save_folder = '/home/osh/data/vlabhufs.ipdisk.co.kr/VOL1/Storage1/Data/SNUBH_Breast/201~300/hdf5_201-300' # 저장될 폴더 경로 설정
hdf5_file_name = f'{sub_folder_name}_data.h5' # 저장될 HDF5 파일 이름 설정

with h5py.File(os.path.join(save_folder, hdf5_file_name), 'w') as hdf5_file:
    with concurrent.futures.ProcessPoolExecutor(max_workers=30) as executor:
        # 각 환자 폴더에 대한 작업을 시작합니다.
        futures_to_folder = {executor.submit(process_patient_folder, patient_folder_name): patient_folder_name for patient_folder_name in patient_folder_names}

        for future in concurrent.futures.as_completed(futures_to_folder):
            patient_folder_name = futures_to_folder[future]
            try:
                patient_num, roi_precontrast = future.result()
                # roi_precontrast를 해당 환자의 번호로 HDF5 파일에 저장
                hdf5_file.create_dataset(str(patient_num), data=roi_precontrast, chunks=True)
            except Exception as e:
                print(f"Failed processing folder: {patient_folder_name} with error: {e}")

Processing folder: 276

Processing folder: 209Processing folder: 224Processing folder: 264

Processing folder: 295Processing folder: 206Processing folder: 230Processing folder: 202Processing folder: 221Processing folder: 254Processing folder: 244Processing folder: 255Processing folder: 213Processing folder: 211Processing folder: 228Processing folder: 298Processing folder: 283Processing folder: 236
Processing folder: 287Processing folder: 239Processing folder: 278Processing folder: 270Processing folder: 251Processing folder: 282Processing folder: 269Processing folder: 253Processing folder: 241Processing folder: 250Processing folder: 292























Processing folder: 208
Processing folder: 231
Processing folder: 285
Processing folder: 271
Processing folder: 237
Processing folder: 234
Processing folder: 243
Processing folder: 219
Processing folder: 267
Processing folder: 215
Processing folder: 272
Processing folder: 274
Processing folder: 297
Processing folder: 210
Processing 

/tmp/ipykernel_31951/301573698.py:31: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  lesion_mask = np.any([np.all(images == color, axis=-1) for color in lesion_colors], axis=0)


Failed processing folder: 227
Processing folder: 216
Failed processing folder: 227 with error: Connectivity for 0D image should be in [1, ..., 0]. Got 0.
Processing folder: 281
Processing folder: 263
Processing folder: 232
Processing folder: 291
Processing folder: 265
Processing folder: 229
Processing folder: 259
Processing folder: 235
Processing folder: 214
Processing folder: 242
Processing folder: 205
Processing folder: 293
Processing folder: 288Processing folder: 277

Processing folder: 289
Processing folder: 275
Processing folder: 203
Processing folder: 258
Processing folder: 233
Processing folder: 212
Processing folder: 260
Processing folder: 240
Processing folder: 207
Processing folder: 225
Processing folder: 226
Processing folder: 223
Processing folder: 256
Processing folder: 247
Processing folder: 217
Processing folder: 220
Processing folder: 280
Processing folder: 261
Processing folder: 204
Processing folder: 245
Processing folder: 201
Processing folder: 252
Processing folder:

In [4]:
import h5py

# 저장된 HDF5 파일 열기
with h5py.File(os.path.join(save_folder, hdf5_file_name), 'r') as hdf5_file:
   
    # 파일 내에 저장된 모든 데이터셋의 이름 출력
    for key in hdf5_file.keys():
        print(key)
        
        # 각 데이터셋의 shape 출력
        print(hdf5_file[key].shape)

201
(100, 104, 115)
202
(113, 167, 179)
203
(92, 133, 184)
204
(106, 86, 45)
205
(78, 28, 110)
206
(16, 23, 28)
207
(51, 124, 108)
208
(42, 49, 46)
209
(92, 149, 155)
210
(108, 103, 200)
211
(32, 54, 64)
212
(70, 134, 126)
213
(31, 19, 61)
214
(70, 81, 71)
215
(44, 28, 63)
216
(36, 39, 36)
217
(88, 188, 115)
218
(13, 24, 23)
219
(13, 29, 24)
220
(10, 32, 36)
221
(49, 38, 116)
222
(60, 71, 72)
223
(53, 92, 123)
224
(29, 33, 38)
225
(76, 51, 143)
226
(14, 42, 36)
228
(92, 141, 128)
229
(59, 27, 89)
230
(58, 40, 91)
231
(33, 59, 29)
232
(51, 30, 119)
233
(97, 155, 320)
234
(16, 28, 27)
235
(99, 203, 248)
236
(43, 28, 99)
237
(23, 40, 34)
238
(73, 87, 68)
239
(99, 82, 123)
240
(42, 70, 51)
241
(38, 52, 37)
242
(43, 32, 82)
243
(29, 17, 65)
244
(58, 68, 82)
245
(15, 30, 27)
246
(138, 48, 179)
247
(93, 45, 109)
248
(21, 32, 22)
249
(104, 135, 130)
250
(34, 34, 52)
251
(10, 33, 40)
252
(35, 24, 61)
253
(19, 47, 53)
254
(70, 33, 113)
255
(68, 120, 48)
256
(87, 45, 128)
257
(126, 91, 116)
258
(